#### Initialize Question Generation Crew

Next we'll create a helper function to append labeled question-sql pairs to our .csv file.

In [ ]:
import os
import csv
def append_question_sql_pair(csv_file_path, question, sql_query):
    """
    Append a question and its corresponding SQL query to a CSV file.
    
    Parameters:
    csv_file_path (str): Path to the CSV file
    question (str): The question text
    sql_query (str): The corresponding SQL query
    
    Returns:
    bool: True if successful, False otherwise
    """
    try:
        # Check if file exists to determine if we need to write headers
        file_exists = os.path.isfile(csv_file_path)
        
        with open(csv_file_path, 'a', newline='', encoding='utf-8') as csvfile:
            csv_writer = csv.writer(csvfile)
            
            # Write headers if file is newly created
            if not file_exists:
                csv_writer.writerow(['Question', 'SQL Query'])
            
            # Write the question-SQL pair
            csv_writer.writerow([question, sql_query])
        
        print(f"Successfully appended to {csv_file_path}")
        return True
    
    except Exception as e:
        print(f"Error appending to CSV: {e}")
        return False

We'll also create a helper to parse out the response, using the `<question>` tag.

In [ ]:
import re
def extract_question(text):
    """
    Extract content between <question> and </question> tags using regex.
    
    Parameters:
    text (str): The input text containing question tags
    
    Returns:
    str: The extracted question content, or None if no match found
    """
    pattern = r'<question>(.*?)</question>'
    # Use re.DOTALL to make '.' match newlines as well
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        # Return the content between the tags, stripping leading/trailing whitespace
        return match.group(1).strip()
    else:
        return None

Now we'll build the input for a random HR topic, along with the DDL for the database.
First will create a list of question categories that we can use to randomly seed for question generation.

In [ ]:
hr_question_categories = [
    "Employee Demographics",
    "Organizational Structure",
    "Headcount Analysis",
    "Location Distribution",
    "Compensation Analysis",
    "Employee Retention",
    "Career Progression",
    "Leave Management",
    "Training Compliance",
    "Skill Development",
    "Benefits Utilization",
    "Performance Trends",
    "Remote Work Analysis",
    "Department Cost Analysis",
    "Salary Banding",
    "Promotion Equity",
    "Succession Planning",
    "Time-Off Impact",
    "Retirement Readiness",
    "Location Strategy",
    "Department Comparative Analysis",
    "Employee Lifecycle Analysis",
    "Job Level Distribution",
    "Cost Center Efficiency",
    "Management Span Assessment",
]

In [ ]:
import random
ddl_file_path = "../module_2/ddl/01_ddl_hr_db.sql"
crew_inputs = {}
try:
    with open(ddl_file_path, 'r') as file:
        ddl_content = file.read()
    
    # Assign to the crew_inputs dictionary
    crew_inputs = {
        "topic": random.choice(hr_question_categories),
        "ddl": ddl_content
    }
    
    print(f"Successfully loaded DDL content. Selected topic: {crew_inputs['topic']}")
    
except FileNotFoundError:
    print(f"Error: The file {ddl_file_path} was not found.")
    # Set empty string or some default value if file not found
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
from crew_question_gen import QuestionGenerationCrew

question_gen_crew = QuestionGenerationCrew(
    user='admin',
    host='localhost', 
    password='txt2sqlWorkshop').crew()

In [ ]:
question_gen_results = question_gen_crew.kickoff(inputs=crew_inputs)

In [ ]:
business_question = extract_question(question_gen_results.raw)
print(f"Here's a generated question for the topic '{crew_inputs.get('topic')}':\n{business_question}")

In [ ]:
# add question to crew_inputs
crew_inputs['question'] = business_question

### Build SQL Generation Agents and Tasks

In [ ]:
# for testing
# print(crew_inputs)
crew_inputs = {'topic': 'Retirement Readiness', 'ddl': '-- Core HR Schema\nDROP SCHEMA IF EXISTS hr_core;\nCREATE SCHEMA hr_core;\n\n-- Employee and Organization Structure\nCREATE TABLE hr_core.locations (\n    location_id INT PRIMARY KEY,\n    address_line1 VARCHAR(100),\n    address_line2 VARCHAR(100),\n    city VARCHAR(50),\n    state VARCHAR(50),\n    country VARCHAR(50),\n    is_remote BOOLEAN,\n    location_type VARCHAR(20),\n    INDEX idx_city_country (city, country),\n    INDEX idx_location_type (location_type)\n);\n\nCREATE TABLE hr_core.employee (\n    employee_id INT PRIMARY KEY,\n    first_name VARCHAR(50),\n    last_name VARCHAR(50),\n    email VARCHAR(100) UNIQUE,\n    hire_date DATETIME,\n    is_active BOOLEAN DEFAULT true,\n    current_location_id INT,\n    FOREIGN KEY (current_location_id) REFERENCES hr_core.locations(location_id),\n    INDEX idx_name (last_name, first_name),\n    INDEX idx_hire_date (hire_date),\n    INDEX idx_is_active (is_active),\n    INDEX idx_current_location (current_location_id)\n);\n\nCREATE TABLE hr_core.departments (\n    department_id INT PRIMARY KEY,\n    department_name VARCHAR(50),\n    cost_center VARCHAR(20),\n    INDEX idx_department_name (department_name),\n    INDEX idx_cost_center (cost_center)\n);\n\nCREATE TABLE hr_core.job_titles (\n    job_title_id INT PRIMARY KEY,\n    title_name VARCHAR(50),\n    job_level INT,\n    job_family VARCHAR(50),\n    INDEX idx_title_name (title_name),\n    INDEX idx_job_level (job_level),\n    INDEX idx_job_family (job_family)\n);\n\n-- Employee Position History\nCREATE TABLE hr_core.employee_positions (\n    position_id INT PRIMARY KEY,\n    employee_id INT,\n    department_id INT,\n    job_title_id INT,\n    manager_id INT,\n    start_date DATE,\n    end_date DATE,\n    salary DECIMAL(12,2),\n    FOREIGN KEY (employee_id) REFERENCES hr_core.employee(employee_id),\n    FOREIGN KEY (department_id) REFERENCES hr_core.departments(department_id),\n    FOREIGN KEY (job_title_id) REFERENCES hr_core.job_titles(job_title_id),\n    FOREIGN KEY (manager_id) REFERENCES hr_core.employee(employee_id),\n    INDEX idx_employee_id (employee_id),\n    INDEX idx_department_id (department_id),\n    INDEX idx_job_title_id (job_title_id),\n    INDEX idx_manager_id (manager_id),\n    INDEX idx_dates (start_date, end_date),\n    INDEX idx_salary (salary)\n);\n\n-- Time Off Schema\nDROP SCHEMA IF EXISTS hr_timeoff;\nCREATE SCHEMA hr_timeoff;\n\nCREATE TABLE hr_timeoff.leave_types (\n    leave_type_id INT PRIMARY KEY,\n    leave_type_name VARCHAR(50),\n    is_paid BOOLEAN,\n    INDEX idx_leave_type_name (leave_type_name),\n    INDEX idx_is_paid (is_paid)\n);\n\nCREATE TABLE hr_timeoff.leave_requests (\n    leave_request_id INT PRIMARY KEY,\n    employee_id INT,\n    leave_type_id INT,\n    start_date DATE,\n    end_date DATE,\n    status VARCHAR(20),\n    approved_by INT,\n    FOREIGN KEY (employee_id) REFERENCES hr_core.employee(employee_id),\n    FOREIGN KEY (leave_type_id) REFERENCES hr_timeoff.leave_types(leave_type_id),\n    FOREIGN KEY (approved_by) REFERENCES hr_core.employee(employee_id),\n    INDEX idx_employee_id (employee_id),\n    INDEX idx_leave_type_id (leave_type_id),\n    INDEX idx_dates (start_date, end_date),\n    INDEX idx_status (status),\n    INDEX idx_approved_by (approved_by)\n);\n\n-- Training Schema\nDROP SCHEMA IF EXISTS hr_training;\nCREATE SCHEMA hr_training;\n\nCREATE TABLE hr_training.courses (\n    course_id INT PRIMARY KEY,\n    course_name VARCHAR(100),\n    description TEXT,\n    duration_hours INT,\n    INDEX idx_course_name (course_name),\n    INDEX idx_duration_hours (duration_hours)\n);\n\nCREATE TABLE hr_training.employee_courses (\n    employee_id INT,\n    course_id INT,\n    completion_date DATE,\n    certification_id VARCHAR(50),\n    PRIMARY KEY (employee_id, course_id),\n    FOREIGN KEY (employee_id) REFERENCES hr_core.employee(employee_id),\n    FOREIGN KEY (course_id) REFERENCES hr_training.courses(course_id),\n    INDEX idx_completion_date (completion_date),\n    INDEX idx_certification_id (certification_id)\n);\n\n-- Benefits Schema\nDROP SCHEMA IF EXISTS hr_benefits;\nCREATE SCHEMA hr_benefits;\n\nCREATE TABLE hr_benefits.retirement_plans (\n    plan_id INT PRIMARY KEY,\n    plan_name VARCHAR(50),\n    description TEXT,\n    provider VARCHAR(50),\n    INDEX idx_plan_name (plan_name),\n    INDEX idx_provider (provider)\n);\n\nCREATE TABLE hr_benefits.employee_retirement_plans (\n    employee_id INT,\n    plan_id INT,\n    enrollment_date DATE,\n    contribution_percentage DECIMAL(5,2),\n    PRIMARY KEY (employee_id, plan_id),\n    FOREIGN KEY (employee_id) REFERENCES hr_core.employee(employee_id),\n    FOREIGN KEY (plan_id) REFERENCES hr_benefits.retirement_plans(plan_id),\n    INDEX idx_enrollment_date (enrollment_date),\n    INDEX idx_contribution_percentage (contribution_percentage)\n);\n\n-- Promotions Schema\nDROP SCHEMA IF EXISTS hr_promotions;\nCREATE SCHEMA hr_promotions;\n\nCREATE TABLE hr_promotions.promotion_types (\n    promotion_type_id INT PRIMARY KEY,\n    type_name VARCHAR(50),\n    description TEXT,\n    INDEX idx_type_name (type_name)\n);\n\nCREATE TABLE hr_promotions.promotions (\n    promotion_id INT PRIMARY KEY,\n    employee_id INT,\n    promotion_type_id INT,\n    old_position_id INT,\n    new_position_id INT,\n    promotion_date DATE,\n    salary_increase_percentage DECIMAL(5,2),\n    FOREIGN KEY (employee_id) REFERENCES hr_core.employee(employee_id),\n    FOREIGN KEY (promotion_type_id) REFERENCES hr_promotions.promotion_types(promotion_type_id),\n    FOREIGN KEY (old_position_id) REFERENCES hr_core.employee_positions(position_id),\n    FOREIGN KEY (new_position_id) REFERENCES hr_core.employee_positions(position_id),\n    INDEX idx_employee_id (employee_id),\n    INDEX idx_promotion_type_id (promotion_type_id),\n    INDEX idx_position_ids (old_position_id, new_position_id),\n    INDEX idx_promotion_date (promotion_date),\n    INDEX idx_salary_increase (salary_increase_percentage)\n);\n\n-- Reporting Schema\nDROP SCHEMA IF EXISTS hr_reporting;\nCREATE SCHEMA hr_reporting;\n\n-- Employee Directory Report\nCREATE TABLE hr_reporting.employee_directory (\n    employee_id INT,\n    full_name VARCHAR(100),\n    email VARCHAR(100),\n    department_name VARCHAR(50),\n    job_title VARCHAR(50),\n    manager_name VARCHAR(100),\n    location_city VARCHAR(50),\n    location_country VARCHAR(50),\n    is_remote BOOLEAN,\n    hire_date DATETIME,\n    current_salary DECIMAL(12,2),\n    INDEX idx_employee_id (employee_id),\n    INDEX idx_full_name (full_name),\n    INDEX idx_email (email),\n    INDEX idx_department_name (department_name),\n    INDEX idx_job_title (job_title),\n    INDEX idx_location (location_city, location_country),\n    INDEX idx_hire_date (hire_date),\n    INDEX idx_salary (current_salary)\n);\n\n-- Leave Balance Report\nCREATE TABLE hr_reporting.leave_summary (\n    employee_id INT,\n    full_name VARCHAR(100),\n    department_name VARCHAR(50),\n    leave_type_name VARCHAR(50),\n    total_days_taken INT,\n    last_leave_date DATE,\n    pending_requests INT,\n    INDEX idx_employee_id (employee_id),\n    INDEX idx_full_name (full_name),\n    INDEX idx_department_name (department_name),\n    INDEX idx_leave_type (leave_type_name),\n    INDEX idx_last_leave_date (last_leave_date)\n);\n\n-- Training Compliance Report\nCREATE TABLE hr_reporting.training_compliance (\n    employee_id INT,\n    full_name VARCHAR(100),\n    department_name VARCHAR(50),\n    job_title VARCHAR(50),\n    total_courses_completed INT,\n    last_completion_date DATE,\n    certifications_count INT,\n    INDEX idx_employee_id (employee_id),\n    INDEX idx_full_name (full_name),\n    INDEX idx_department_name (department_name),\n    INDEX idx_job_title (job_title),\n    INDEX idx_last_completion_date (last_completion_date),\n    INDEX idx_courses_completed (total_courses_completed)\n);\n\n-- Compensation History Report\nCREATE TABLE hr_reporting.compensation_history (\n    employee_id INT,\n    full_name VARCHAR(100),\n    department_name VARCHAR(50),\n    job_title VARCHAR(50),\n    salary_amount DECIMAL(12,2),\n    previous_salary DECIMAL(12,2),\n    change_percentage DECIMAL(5,2),\n    change_date DATE,\n    change_reason VARCHAR(50),\n    INDEX idx_employee_id (employee_id),\n    INDEX idx_full_name (full_name),\n    INDEX idx_department_name (department_name),\n    INDEX idx_job_title (job_title),\n    INDEX idx_salary (salary_amount),\n    INDEX idx_change_date (change_date),\n    INDEX idx_change_reason (change_reason)\n);\n\n-- Department Headcount Report\nCREATE TABLE hr_reporting.department_metrics (\n    department_id INT,\n    department_name VARCHAR(50),\n    total_employees INT,\n    avg_salary DECIMAL(12,2),\n    avg_tenure_years DECIMAL(5,2),\n    remote_employee_count INT,\n    open_positions INT,\n    INDEX idx_department_id (department_id),\n    INDEX idx_department_name (department_name),\n    INDEX idx_total_employees (total_employees),\n    INDEX idx_avg_salary (avg_salary)\n);\n\n-- Benefits Enrollment Summary\nCREATE TABLE hr_reporting.benefits_summary (\n    employee_id INT,\n    full_name VARCHAR(100),\n    department_name VARCHAR(50),\n    retirement_plan_name VARCHAR(50),\n    contribution_percentage DECIMAL(5,2),\n    enrollment_date DATE,\n    total_benefits_cost DECIMAL(12,2),\n    INDEX idx_employee_id (employee_id),\n    INDEX idx_full_name (full_name),\n    INDEX idx_department_name (department_name),\n    INDEX idx_retirement_plan (retirement_plan_name),\n    INDEX idx_enrollment_date (enrollment_date),\n    INDEX idx_benefits_cost (total_benefits_cost)\n);', 'question': 'For employees currently active in the system, what is the retirement plan participation analysis by department for those who enrolled in retirement plans on or after January 1, 2019? Include:\n\n1. Department name\n2. Total count of enrolled active employees in department\n3. Average contribution percentage (as decimal between 0-100)\n4. Average current base salary (from most recent position)\n5. Retirement plan provider name and plan name\n\nDisplay only departments with at least 5 enrolled employees.\nSort results by average contribution percentage in descending order.'}

In [ ]:
from crew_sql_gen import SQLGenerationCrew

sql_gen_crew = SQLGenerationCrew(
    user='admin',
    host='localhost', 
    password='txt2sqlWorkshop').crew()

In [ ]:
sql_gen_results = sql_gen_crew.kickoff(inputs=crew_inputs)